In [1]:
""" KR Example
This notebook demonstrates:
1. How to represent knowledge as:
-Facts
-Rules
2. How to perform simple inference:
-Forward chaining: Repeatedly apply rules to know facts to derive new facts.

Why this is "knowledge representation"?
- We are't storing raw text like "John is a parent of Mary"
- We are representing knowledge in a structured format so a computer can reason over it

"""




' KR Example\nThis notebook demonstrates:\n1. How to represent knowledge as:\n-Facts\n-Rules\n2. How to perform simple inference:\n-Forward chaining: Repeatedly apply rules to know facts to derive new facts.\n\nWhy this is "knowledge representation"?\n- We are\'t storing raw text like "John is a parent of Mary"\n- We are representing knowledge in a structured format so a computer can reason over it\n\n'

In [5]:
from dataclasses import dataclass
from typing import Tuple, List, Callable, Set 

# A fact is a triple (subject, relation, object)
Fact = Tuple[str, str, str]

#Rule structure: a name of a fxn that derives facts from existing ones
@dataclass(frozen=True)
class Rule:
    name: str
    func: Callable[[Set[Fact]], Set[Fact]]

class KnowledgeBase:
    def __init__(self):
        self.facts: Set[Fact] = set()#known facts
        self.rules: List[Rule] = []#inference rules

    def add_fact(self, fact: Fact):
        # Add a fact to the knowledge base(duplicate facts are ignored)
        self.facts.add(fact)

    def add_rule(self, rule: Rule):
        # Add a reasoning rule to the knowledge base
        self.rules.append(rule)

    def infer(self):
        # Apply rules repeatedly until no new facts can be found/derived
        while True:
            new_facts = set()
            for rule in self.rules:
                inferred = rule.func(self.facts)
                new_facts|= (inferred - self.facts)

            if not new_facts:
                break # stop if nothing new is found/inferred
            self.facts |= new_facts

    def query(self, subject=None, relation=None, obj=None):
            # Simple pattern-based query
        return [
            f for f in self.facts
            if (subject is None or f[0] == subject) and
                (relation is None or f[1] == relation) and
                (obj is None or f[2] == obj)
            ]


# RULE DEFINITIONS
def parent_implies_child(fact: Set[Fact]) -> Set[Fact]:
    # If X is parent of Y, then Y is child of X
    return {(o,"child_of",s) for (s,r,o) in fact if r=="parent_of"}

def parent_implies_ancestor(fact: Set[Fact]) -> Set[Fact]:
    # If X is parent of Y, then X is ancestor of Y
    return {(s,"ancestor_of",o) for (s,r,o) in fact if r=="parent_of"}

def ancestor_transitive(fact: Set[Fact]) -> Set[Fact]:
    # If X ancestor of Y and Y ancestor of Z, then X ancestor of Z 
    ancestors = [(s,o) for (s,r,o) in fact if r=="ancestor_of"]   
    return {
        (x, "ancestor_of", z)
        for (x,y) in ancestors  
        for (y2,z) in ancestors
        if y == y2 and x != z
    }


# BUILD KNOWLEDGE BASE

kb = KnowledgeBase()      

# Base facts
kb.add_fact(("Alice","parent_of","Bob"))
kb.add_fact(("Bob","parent_of","Charlie"))
kb.add_fact(("Diana","parent_of","Eve"))

# Add rules
kb.add_rule(Rule("parent -> child", parent_implies_child))
kb.add_rule(Rule("parent -> ancestor", parent_implies_ancestor))
kb.add_rule(Rule("ancestor transitive", ancestor_transitive))

# Perform inference
kb.infer()


# QUERIES
print("All Facts:")
for fact in sorted(kb.facts):
    print(fact) 

All Facts:
('Alice', 'ancestor_of', 'Bob')
('Alice', 'ancestor_of', 'Charlie')
('Alice', 'parent_of', 'Bob')
('Bob', 'ancestor_of', 'Charlie')
('Bob', 'child_of', 'Alice')
('Bob', 'parent_of', 'Charlie')
('Charlie', 'child_of', 'Bob')
('Diana', 'ancestor_of', 'Eve')
('Diana', 'parent_of', 'Eve')
('Eve', 'child_of', 'Diana')
